## 15. Saving plots

### Basic example on how to save plots with plotly. Note that the "kaleido" package in included as a dependency for RNApysoforms, therefore you are able to save images as ".png", ".pdf", and etc without having to install extra packages.

### Kaleido has an unresolved bug that makes the saving process hang forever and never finish on some machines. If that is happening to you see this issue for guidance on how to deal with it: https://github.com/plotly/Kaleido/issues/134

In [12]:
import RNApysoforms as RNApy

## Add import statement for kaleido
import kaleido 

In [13]:
## Path to your ENSEMBL GTF file, counts matrix file, and metadata file
ensembl_gtf_path = "../../dash_apps/RNApysoforms/tests/test_data/Homo_sapiens_chr21_and_Y.GRCh38.110.gtf"
counts_matrix_path = "../../dash_apps/RNApysoforms/tests/test_data/counts_matrix_chr21_and_Y.tsv"
metadata_path = "../../dash_apps/RNApysoforms/tests/test_data/sample_metadata.tsv"

## Process annotation and expression matrix
annotation = RNApy.read_ensembl_gtf(ensembl_gtf_path)
expression_matrix = RNApy.read_expression_matrix(expression_matrix_path=counts_matrix_path,
                                          metadata_path=metadata_path,
                                           cpm_normalization=True, relative_abundance=True)

## Add transcript name
expression_matrix = expression_matrix.join(annotation[["transcript_id", "transcript_name"]].clone(), 
                                                   on="transcript_id", how="inner")

## Filter APP data
app_annotation, app_expression_matrix = RNApy.gene_filtering(annotation=annotation, expression_matrix=expression_matrix, target_gene="APP",
                                                        order_by_expression=True, keep_top_expressed_transcripts=5,
                                                        order_by_expression_column="counts", transcript_id_column="transcript_name")

# Rescale introns
app_annotation = RNApy.shorten_gaps(app_annotation)


## Make traces for APP figures
traces = RNApy.make_traces(annotation=app_annotation,  expression_matrix=app_expression_matrix,
                            x_start="rescaled_start", x_end="rescaled_end",
                            y='transcript_name', annotation_hue="transcript_biotype",
                            hover_start="start", hover_end="end",
                            expression_columns=["counts", "relative_abundance"],
                            expression_hue="AD status", marker_size=1, arrow_size=6,
                            show_box_mean=False)

## Make figure
fig = RNApy.make_plot(traces=traces, subplot_titles=["Transcript Structure", "Counts", "Relative Abundance (%)"], 
                   width=1200, height=500)




## Show figure
fig.show()


## Save figure as HTML, PDF, and PNG

# Save as HTML
fig.write_html("./RNApysoforms.html")

# Save as PDF
fig.write_image("./RNApysoforms.pdf", format="pdf", engine="kaleido", width=1200, height=500)

# Save as PNG
fig.write_image("./RNApysoforms.png", format="png", engine="kaleido", width=1200, height=500)

### You can click on the legend items to make figure elements appear and disappear.

### The legend title will get grayed out when clicking on the first legend item. I could not find a workaround for that with the current plotly release (version 5).

### The hovering for exons and CDS works best if you hover your mouse over the edges of the CDS/exon boxes.